In [7]:
import theano
import theano.tensor as T
import numpy as np

In [15]:
class layer:
    def __init__(self, activation_func, W_init, b_init):
        '''
        output = activation(WX+b)
        X is the data matrix (each column is one example)
        W takes 'n_in' inputs from the previous layer (the number of previous layers nodes)
        and has 'n_out' outputs (the number of the next layers nodes)
        
        parameters:
        W_init: initial weight values
            np.ndarray
        b_init: initial bias value
            np.ndarray
        activation_func: activation function for output of layer
            theano.tensor.elemwise.Elemwise
        '''
        n_out, n_in = W_init.shape
        self.W = theano.shared(W_init.astype(theano.config.floatX), borrow=True)
        #borrow=True as we allow Theano to use memory for this object (make faster)
        self.b = theano.shared(b_init.reshape(n_out, 1).astype(theano.config.floatX), borrow=True, 
                               broadcastable=(False, True))
        self.activation_func = activation_func
        self.params = [self.W, self.b]
        
    def output(self, X):
        '''
        Gives the output: activation(WX+b)
        '''
        pre_activation = T.dot(self.W, X) + self.b
        return(self.activation_func(pre_activation))

In [ ]:
class mlp:
    def __init__(self, n_layer, output_activation_func, W_init, b_init):
        '''
        output of sequential layers
        
        parameters:
        n_layer: number of layers
            np.ndarray
        output_activation_func: activation function of the output layer
        '''
        self.layers = [layer(activation_func, W_init, b_init) for i in range(n_layer-1)]
        self.layers.append(layer(output_activation_func, W_init, b_init))
            
        self.params = [layer.params for layer in self.layers]
    
    def output(self, X):
        for layer in self.layers:
            X = layer.output(X)
        return(X)
    
    def error(self, X, y):
        return(T.sum((y-self.output(X))**2))